###Riyad Bin Rafiq
###03/25/2022

## Part A

In [ ]:
import os
import torch
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms 
import cv2
import glob
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/PhD courses/CSCE 5218 Deep Learning/Assignments/Assignment 1

/content/drive/My Drive/PhD courses/CSCE 5218 Deep Learning/Assignments/Assignment 1


In [ ]:
!7z x 'cifar10.zip'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 147491716 bytes (141 MiB)

Extracting archive: cifar10.zip
  6% 4096 Open              --
Path = cifar10.zip
Type = zip
Physical Size = 147491716

  0%    
Would you like to replace the existing file:
  Path:     ./cifar10_train/frog/frog_00000.png
  Size:     2461 bytes (3 KiB)
  Modified: 2022-03-24 17:39:56
with the file from archive:
  Path:     cifar10_train/frog/frog_00000.png
  Size:     2461 bytes (3 KiB)
  Modified: 2020-02-08 01:02:05
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? 

In [ ]:
!ls

'Assignment 1.ipynb'   cifar10_test   cifar10_train   cifar10.zip


In [ ]:
''' This function is used to store the image's paths and class names in two separate lists  '''

def image_path_class_store(full_image_path):
  image_path = []
  classes = []
  for data_path in glob.glob(full_image_path + '/*'):
    classes.append(data_path.split('/')[-1]) 
    image_path.append(glob.glob(data_path + '/*'))
  image_path_list = [item for sublist in image_path for item in sublist]
  return image_path_list, classes

In [ ]:
''' The above function is used to separate paths and class names for train and test dataset '''

train_image_path, train_classes = image_path_class_store('cifar10_train')
test_image_path, _ = image_path_class_store('cifar10_test')

In [ ]:
print(train_image_path)
print(test_image_path)
print(train_classes)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
''' Each class has been assigned to a numeric label '''

class_to_label = dict(frog=0, truck=1, deer=2, bird=3, horse=4, ship=5, cat=6, dog=7, automobile=8, airplane=9)

In [ ]:
''' The CifarDataset is a custom dataset that has been used for preparaing the train and test data
    based on the passed arguments: path of image directory and image transformation. 
'''

class CifarDataset(Dataset):
    def __init__(self, image_paths, transform):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
      image_filepath = self.image_paths[idx]
      image = cv2.imread(image_filepath)
      image_class = image_filepath.split('/')[-2]
      label = class_to_label[image_class]
      image = self.transform(image)
      return image, label

In [ ]:
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), transforms.Grayscale(num_output_channels=1)])

In [ ]:
train_dataset = CifarDataset(train_image_path, transforms)
test_dataset = CifarDataset(test_image_path, transforms)

In [ ]:
train_dataset[0][0].shape

torch.Size([1, 32, 32])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset,batch_size=128, shuffle=False, num_workers=2)

## Part B

In [ ]:
''' This is 3-layer MLP model (one input layer, one hidden layer and one output layer). '''

class MultiLayerPerceptronModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 512)
        self.output_fc = nn.Linear(512, output_dim)

    def forward(self, x):
        
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        h = F.relu(self.input_fc(x))
        y_pred = self.output_fc(h)

        return y_pred

In [ ]:
INPUT_DIM = 32 * 32
OUTPUT_DIM = 10

model = MultiLayerPerceptronModel(INPUT_DIM, OUTPUT_DIM)

In [ ]:
print(model)

MultiLayerPerceptronModel(
  (input_fc): Linear(in_features=1024, out_features=512, bias=True)
  (output_fc): Linear(in_features=512, out_features=10, bias=True)
)


In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
''' This train function is used to train the multilayer perceptron for specific number of epochs. '''
PATH = './cifar_net.pth'

def train(epoch, model, loader, optimizer, criterion, device):

    model.train()
  
    for iter in range(epoch):
      
      epoch_loss = 0
      epoch_acc = 0
      
      for i, data in enumerate(loader, 0):
        
        batch_loss = 0
        batch_acc = 0

        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        acc = calculate_accuracy(outputs, labels)
        loss.backward()
        optimizer.step()
        batch_loss += loss.item()
        batch_acc += acc.item()
        print('Batch Loss: ')
        print(f'\tTrain Loss: {batch_loss/len(inputs):.3f} | Train Acc: {(batch_acc/len(inputs))*100:.2f}%')
        epoch_loss += loss.item()
        epoch_acc += acc.item()
      print(f'Epoch: {iter+1:02}')
      print(f'\tFinal train Loss: {epoch_loss/len(loader):.3f} | Final train Acc: {(epoch_acc/len(loader))*100:.2f}%')
    # Saving the model 
    torch.save(model.state_dict(), PATH)

In [ ]:
''' This function evaluate the trained model on the test set. '''

def evaluation(model, loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for data in loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
EPOCHS = 1

# Calling train function to train the network. It returns the final training loss and accuracy. 
train(EPOCHS, model, train_loader, optimizer, criterion, device)
trained_model = model
trained_model = trained_model.to(device)
# Load the trained model
trained_model.load_state_dict(torch.load(PATH))
# Evaluatet the trained model on the test set
evaluation(trained_model, test_loader)


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f38f1228710>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f38f1228710>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

Batch Loss: 
	Train Loss: 0.012 | Train Acc: 0.37%
Batch Loss: 
	Train Loss: 0.012 | Train Acc: 0.37%
Batch Loss: 
	Train Loss: 0.013 | Train Acc: 0.32%
Batch Loss: 
	Train Loss: 0.012 | Train Acc: 0.36%
Batch Loss: 
	Train Loss: 0.012 | Train Acc: 0.33%
Batch Loss: 
	Train Loss: 0.012 | Train Acc: 0.33%
Batch Loss: 
	Train Loss: 0.012 | Train Acc: 0.39%
Batch Loss: 
	Train Loss: 0.011 | Train Acc: 0.42%
Batch Loss: 
	Train Loss: 0.011 | Train Acc: 0.42%
Batch Loss: 
	Train Loss: 0.012 | Train Acc: 0.37%
Batch Loss: 
	Train Loss: 0.012 | Train Acc: 0.37%
Batch Loss: 
	Train Loss: 0.013 | Train Acc: 0.36%
Batch Loss: 
	Train Loss: 0.011 | Train Acc: 0.42%
Batch Loss: 
	Train Loss: 0.012 | Train Acc: 0.37%
Batch Loss: 
	Train Loss: 0.010 | Train Acc: 0.43%
Batch Loss: 
	Train Loss: 0.011 | Train Acc: 0.41%
Batch Loss: 
	Train Loss: 0.012 | Train Acc: 0.37%
Batch Loss: 
	Train Loss: 0.012 | Train Acc: 0.37%
Batch Loss: 
	Train Loss: 0.011 | Train Acc: 0.42%
Batch Loss: 
	Train Loss: 0.012